In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
!pip install openpyxl

In [3]:
train_cov=pd.read_excel("../input/covid19-radiography-database/COVID-19_Radiography_Dataset/COVID.metadata.xlsx")
train_cov["label"]= "COVID"
train_nor=pd.read_excel("../input/covid19-radiography-database/COVID-19_Radiography_Dataset/Normal.metadata.xlsx").sample(4192)
train_nor["label"]= "NORMAL"
train_pneu=pd.read_excel("../input/covid19-radiography-database/COVID-19_Radiography_Dataset/Viral Pneumonia.metadata.xlsx")
train_pneu["label"]= "PNEUMONIA"

In [4]:
import os
import pandas as pd
y = 0
data = []
for dirname, _, filenames in os.walk('../input/chest-xray-covid19-pneumonia/Data/train/PNEUMONIA'):
    for filename in filenames:
        y = y + 1
        if y<=2847:
            data.append(os.path.join(dirname, filename))


In [5]:
df_pneu2 = pd.DataFrame(data)

In [6]:
df_pneu2.rename(columns = {0:'images'}, inplace = True)
df_pneu2["target"] = "PNEUMONIA"

In [7]:
df_pneu2

In [8]:
import os
import pandas as pd
data1 = []
x = 0
for dirname, _, filenames in os.walk('../input/chest-xray-covid19-pneumonia/Data/train/COVID19'):
    for filename in filenames:
            data1.append(os.path.join(dirname, filename))
for dirname, _, filenames in os.walk('../input/chest-xray-covid19-pneumonia/Data/test/COVID19'):
    for filename in filenames:
            data1.append(os.path.join(dirname, filename))

In [9]:
df_cov2 = pd.DataFrame(data1)
df_cov2.rename(columns = {0:'images'}, inplace = True)
df_cov2["target"] = "COVID"

In [10]:
df_cov2.shape

In [11]:
train_dir='../input/covid19-radiography-database/COVID-19_Radiography_Dataset/COVID/'
data=[]
labels=[]
for i in range(train_cov.shape[0]):
    data.append(train_dir+train_cov['FILE NAME'].iloc[i]+'.png')
    labels.append(train_cov['label'].iloc[i])
df_cov=pd.DataFrame(data)
df_cov.columns=['images']
df_cov['target']=labels

In [12]:
x= []
for i in range(train_nor.shape[0]):
    x.append(train_nor["FILE NAME"].iloc[i].title())

In [13]:
train_dir='../input/covid19-radiography-database/COVID-19_Radiography_Dataset/Normal/'
data=[]
labels=[]
for i in range(train_nor.shape[0]):
    data.append(train_dir+x[i]+'.png')
    labels.append(train_nor['label'].iloc[i])
df_nor=pd.DataFrame(data)
df_nor.columns=['images']
df_nor['target']=labels

In [14]:
df_nor

In [15]:
train_dir='../input/covid19-radiography-database/COVID-19_Radiography_Dataset/Viral Pneumonia/'
data=[]
labels=[]
for i in range(train_pneu.shape[0]):
    data.append(train_dir+train_pneu['FILE NAME'].iloc[i]+'.png')
    labels.append(train_pneu['label'].iloc[i])
df_pneu=pd.DataFrame(data)
df_pneu.columns=['images']
df_pneu['target']=labels

In [16]:
train=pd.concat([df_pneu,df_nor,df_cov,df_pneu2,df_cov2])
train=train.reset_index()

In [17]:
train["target"].value_counts()

In [18]:
train= train.drop(["index"], axis=1)

In [19]:
train

In [20]:
train.shape

In [21]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(train['images'],train['target'],test_size=0.2,random_state=42)

train_gen = pd.DataFrame({'image_dir': x_train, 'target': y_train})
val_gen = pd.DataFrame({'image_dir': x_val, 'target': y_val})

In [22]:
train_gen

In [23]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,horizontal_flip=True)
val_datagen=ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_dataframe(
    train_gen,
    x_col='image_dir',
    y_col='target',
    target_size=(256, 256),
    batch_size=32,
    shuffle=True,
    class_mode='sparse',
validate_filenames=True)

validation_generator = val_datagen.flow_from_dataframe(
    val_gen,
    x_col='image_dir',
    y_col='target',
    target_size=(256, 256),
    shuffle=False,
    batch_size=32,
    class_mode='sparse',
validate_filenames=True)

In [24]:
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras import layers
from tensorflow.keras.metrics import AUC
from tensorflow.keras.activations import sigmoid
from tensorflow.keras.optimizers import SGD, Adam, Adamax
from tensorflow.keras import Model
import tensorflow as tf

In [25]:
model = tf.keras.models.Sequential([
      layers.BatchNormalization(),
      layers.Conv2D(64, 3, activation='relu'),
      layers.MaxPooling2D(),
      layers.Dropout(0.3),
      layers.Conv2D(128,3,activation='relu'),
      layers.MaxPooling2D(),
      layers.Conv2D(256, 3, activation='relu'),
      layers.MaxPooling2D(),
      layers.Dropout(0.3),
      layers.Conv2D(512, 3, activation='relu'),
      layers.MaxPooling2D(),
      layers.Dropout(0.2),
      layers.Flatten(),
      layers.Dense(1024, activation='relu'),
      layers.Dropout(0.15),
      layers.Dense(3, activation= 'softmax')
    ])

In [26]:
#to avoid overfitting
early = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5)

In [27]:
import tensorflow.keras
#from focal_loss import BinaryFocalLoss
from tensorflow.keras.metrics import AUC
opt = tensorflow.keras.optimizers.Adam(learning_rate = 1e-3)
model.compile(optimizer=opt,loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# # model.compile(loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'],optimizer=opt)
# model.compile(loss=BinaryFocalLoss(gamma=2), metrics=['AUC'],optimizer=opt)

In [28]:
nb_epochs = 50
batch_size=32

In [29]:
device_name = tf.test.gpu_device_name()
if "GPU" not in device_name:
    print("GPU device not found")
print('Found GPU at: {}'.format(device_name))

plt.rcParams['figure.figsize'] = [8.0, 8.0]
plt.rcParams['figure.dpi'] = 100

In [30]:

with tf.device('/gpu:0'):
    print("Training the model with gpu . . .")
    hist=model.fit(
        train_generator,
        epochs=nb_epochs,
        validation_data=validation_generator)
## fit model
# history = model.fit(train_generator,validation_data = validation_generator,epochs = 100,callbacks=callbacks)#callbacks=[early]



In [32]:
model.save("custom_model_50epochs.h5")

# <a href="Your file path"> Download File </a>
<a href="./custom_model_50epochs.h5"> Download File </a>

In [33]:
def plot_accuracy_loss_chart(history):
    epochs = [i for i in range(50)]
    fig , ax = plt.subplots(1,2)
    train_acc = hist.history['accuracy']
    train_loss = hist.history['loss']
    val_acc = hist.history['val_accuracy']
    val_loss = hist.history['val_loss']
    fig.set_size_inches(20,10)
    ax[0].plot(epochs , train_acc , 'go-' , label = 'Training Accuracy')
    ax[0].plot(epochs , val_acc , 'ro-' , label = 'Validation Accuracy')
    ax[0].set_title('Training & Validation Accuracy')
    ax[0].legend()
    ax[0].set_xlabel("Epochs")
    ax[0].set_ylabel("Accuracy")

    ax[1].plot(epochs , train_loss , 'g-o' , label = 'Training Loss')
    ax[1].plot(epochs , val_loss , 'r-o' , label = 'Validation Loss')
    ax[1].set_title('Training & Validation Loss')
    ax[1].legend()
    ax[1].set_xlabel("Epochs")
    ax[1].set_ylabel("Training & Validation Loss")
    plt.show()

In [34]:
plot_accuracy_loss_chart(hist)

In [35]:
s = model.predict(validation_generator)

In [38]:
from sklearn.metrics import confusion_matrix
# Import seaborn
import seaborn as sns

plt.rcParams.update({'font.size': 16})
predictions = model.predict(validation_generator)
predictions = np.argmax(predictions,axis=1)
predictions[:15]
cm = confusion_matrix(validation_generator.classes, predictions)
cm = pd.DataFrame(cm, index = ['0', '1', '2'], columns = ['0', '1', '2'])
cm
class_names = ['COVID','NORMAL','VIRAL PNEUMONIA']
def plot_confusion_matrix (cm):
    plt.figure(figsize = (10,10))
    sns.heatmap(
        cm, 
        cmap = 'Blues', 
        linecolor = 'black', 
        linewidth = 1, 
        annot = True, 
        fmt = '', 
        xticklabels = class_names, 
        yticklabels = class_names)
    
plot_confusion_matrix(cm)

In [39]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
cm = confusion_matrix(validation_generator.classes, predictions, normalize='true')
def plot_confusion_matrix (cm):
    plt.figure(figsize = (8,8))
    sns.heatmap(
        cm, 
        cmap = 'Blues', 
        linecolor = 'black', 
        linewidth = 1, 
        annot = True, 
        fmt=".2%", 
        xticklabels = class_names, 
        yticklabels = class_names)
    
plot_confusion_matrix(cm)

In [40]:
model.save_weights("model.hdf5")

In [46]:
from tensorflow.keras.models import load_model
model = load_model("./output/kaggle/working/model.hdf5")
train_loss, train_acc = model.evaluate(train_generator)
test_loss, test_acc   = model.evaluate(validation_generator)
print("final train accuracy = {:.2f} , validation accuracy = {:.2f}".format(train_acc*100, test_acc*100))